In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import feather
from extras import *
pd.set_option('display.max_columns', None)

In [2]:
ja = Player('Ja Morant', '1629630', '1610612763', ['2019-20','2020-21','2021-22'], 'm/moranja01', ['2020', '2021', '2022'])
drose = Player('Derrick Rose', '201565', '1610612741', ['2008-09','2009-10','2010-11'], 'r/rosede01', ['2009', '2010', '2011'])
west = Player('Russell Westbrook', '201566', '1610612760', ['2008-09','2009-10','2010-11'], 'w/westbru01', ['2009', '2010', '2011'])
iver = Player('Allen Iverson', '947', '1610612755', ['1996-97','1997-98','1998-99'], 'i/iversal01', [ '1997', '1998', '1999'])
players = [ja, drose, west, iver]
players2 = [ja, drose]

Processing data for the total points graph

In [3]:
for player in players2:
    player_data = pd.DataFrame()
    for year in player.bballrefseasons():
        new_season = pd.read_json('../data/'+ player.nbaid() +'/'+ year +'boxscores.json')
        player_data = pd.concat([player_data, new_season], axis=0)
    player_data = player_data.reset_index(drop=True)
    player_data = player_data[['PTS']].cumsum()
    feather.write_dataframe(player_data, './Rdata/'+ player.nbaid() + '_cumpts.feather')

Processing data for radar plots

In [ ]:
ja_season_avg = pd.read_json('../data/1629630/season_avg_per100.json')
ja_season_avg = (
    ja_season_avg.T
    .rename(columns=ja_season_avg['By Year'])
    .drop(['By Year', 'TEAM'], axis=0)
    .reset_index()
)
feather.write_dataframe(ja_season_avg, './Rdata/ja_season_avg_per100.feather')
drose_season_avg = pd.read_json('../data/201565/season_avg_per100.json')

Processing data for beeswarm plots

In [5]:
def getBin(num, q25, q75):
    if num < q25:
        return 'LOW'
    elif (num >= q25) and (num < q75):
        return 'MEDIUM'
    else:
        return 'HIGH'

allyears = pd.DataFrame()

for year in players2[0].nbaseasons():
    drivespg = pd.read_json('../data/1629630/20' + year[-2:] + 'drivespergame.json')
    drivespg.describe()
    q25, q75 = drivespg['DRIVES'].quantile(.25), drivespg['DRIVES'].quantile(.75)
    drivespg['BIN'] = drivespg['DRIVES'].apply(getBin, args=(q25, q75))
    drivespg['X'] = ['']*len(drivespg)
    drivespg['YEAR'] = [year]*len(drivespg)
    allyears = pd.concat([allyears, drivespg], axis=0)

feather.write_dataframe(allyears, './Rdata/drivespg.feather')

Processing data for shot_plots

In [4]:
for player in players:
    allshots = pd.DataFrame()
    for season in player.nbaseasons():
        allshots = pd.concat([allshots, pd.read_json('../data/' + player.nbaid() + '/' + season + '_shots.json')], axis=0)
    feather.write_dataframe(allshots, './Rdata/'+ player.nbaid() +'_allshots.feather')